<a href="https://colab.research.google.com/github/100455376/100455376/blob/main/Extra_Assigment_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exercise: Accelerating Jacobi's Method for Eigenvalue Computation**

Jacobi's method iteratively reduces off-diagonal elements of a symmetric matrix to zero. However, due to numerical effects, previously eliminated off-diagonal elements may reappear, requiring many iterations for full convergence.

To accelerate convergence, we first reduce the matrix to tridiagonal form before applying Jacobi's method. This reduction preserves certain off-diagonal zeros throughout the iterations, improving efficiency.

## **Tasks**

### (i) Implement Tridiagonal Reduction
- Write a Python script that first reduces a given symmetric matrix to tridiagonal form before applying Jacobi's method. Consider implementing Householder reduction.
- Verify that your script correctly maintains the structure of a tridiagonal matrix throughout the iterations.

### (ii) Compare Performance Using Iteration Count
Instead of measuring execution time, compare the **number of iterations required for convergence**, as this provides a more reliable efficiency metric.

1. Implement both:
   - The **standard Jacobi method**.
   - The **tridiagonalized Jacobi method** (applying tridiagonal reduction first).
2. Set the convergence criterion discussed in class (Jacobi's notes, *Section 2.2.3*)

3. Generate a sequence of random symmetric matrices of size $N \times N$ and test both scripts. Count the number of iterations required for each method to meet the convergence criteria.

   ```python
   import numpy as np

   def generate_symmetric_matrix(N):
       A = np.random.rand(N, N)
       return (A + A.T) / 2  # Ensure symmetry

   N_values = [4, 8, 16, 32, 64, 128, 256, ...]
   matrices = [generate_symmetric_matrix(N) for N in N_values]
   ```

### (iii) Analyze and Discuss Results
- Compare how the number of iterations scales with $N$.
- Does tridiagonal reduction significantly speed up convergence?
- How does the efficiency improvement change for large matrices?
- Verify eigenvalues using `numpy.linalg.eigh()`.
- Extend the analysis by plotting the decay of the sum of squared off-diagonal elements over iterations.

In [1]:


# TASK 4
# CODE WITH THE USE OF "RESOURCES" BUT ALWAYS TRYING TO DO WHAT I KNOW, AND COMPLEMENTING OR COMMENTING TO UNDERSTAND WHAT HAS BEEN DONE

import numpy as np # Necessary libraries

def generate_symmetric_matrix(N):#As given in the code
    A = np.random.rand(N, N)
    return (A + A.T) / 2  # Ensure symmetry by averaging A with its transpose

def householder_tridiagonal(A):
    # Householder reduction zeroes out, systematically, all the sub‐/super- diagonal elements except for the first sub‐/super- diagonals. Resulting in a tridiagonal matrix T.
    # Tridiagonal matrix: A tridiagonal matrix, in linear algebra, is a band matrix with non-zero elements only on the main diagonal, the subdiagonal (below the main diagonal), and the superdiagonal (above the main diagonal).
    # https://atozmath.com/example/MatrixEv.aspx?q=qrdecomphh&q1=E1

    # Reduces a symmetric matrix A to tridiagonal form T using Householder reflections.
    # Computes T = Q^T A Q where Q is a product of Householder matrices. Only the tridiagonal part is preserved for this task; the algorithm does not compute Q explicitly.
    # Parameters: A (np.ndarray): A symmetric matrix of shape (n, n). Returns: T (np.ndarray): A tridiagonal matrix similar to A.

    # Work on a copy of A, and ensure type float
    A = A.copy().astype(float)
    n = A.shape[0]

    # Iterate to eliminate the sub-diagonal elements below the first subdiagonal.
    for k in range(n - 2): # We perform n-2 Householder steps
        # First, extract the vector x (elements from row k+1 down in column k)
        x = A[k+1:n, k]
        norm_x = np.linalg.norm(x)

        if norm_x == 0: # If the norm is zero, then the vector is already zero so no reflection  needed
            continue

        sign = -1 if x[0] < 0 else 1 # Determine the sign to avoid cancellation (If x[0] is zero, we choose sign = 1)
        alpha = -sign * norm_x # Reflection scalar alpha

        u = x.copy() # Householder vector: u = x - alpha * e1, where e1 is the first canonical basis vector
        u[0] -= alpha
        norm_u = np.linalg.norm(u) # Normalize u to obtain the Householder vector v

        if norm_u == 0: # Check for norm 0
            continue
        v = u / norm_u

        # Apply the Householder transformation to the remaining submatrix:
        # We update the block A[k+1:n, k+1:n] by:
        #     A_new = A - 2 v (v^T A) - 2 (A v) v^T + 4 v (v^T A v) v^T
        # Symmetric rank-2 update is used since A is symmetric
        # Compute w = 2 * (A[k+1:n, k+1:n] @ v)
        w = 2 * np.dot(A[k+1:n, k+1:n], v)
        # Update the submatrix using a rank-2 modification:
        A[k+1:n, k+1:n] -= np.outer(v, w) + np.outer(w, v)

        # Update the k-th column and row to store the reflector information
        # Off-diagonals outside the tridiagonal band are explicitly zeroed
        A[k+1:n, k] = 0.0
        A[k, k+1:n] = 0.0

        # FInally, store the reflection coefficient in the first subdiagonal
        A[k+1, k] = alpha
        A[k, k+1] = alpha  # Since the matrix is symmetric

    return A


def jacobi_method(A, tol=1e-12, max_iterations=10000):
    # Standard Jacobi method to compute eigenvalues (optionally eigenvectors). Returns A diagonal matrix D with eigenvalues on the diagonal iteration_count
    A = A.copy().astype(float)
    N = A.shape[0]

    def off_diagonal_norm(M):
        return np.sqrt(np.sum(M[np.triu_indices(N, k=1)]**2 + M[np.tril_indices(N, k=-1)]**2))

    iteration_count = 0

    while True:
        iteration_count += 1
        # First find index (p,q) of the largest off-diagonal element
        off_diag_indices = np.triu_indices(N, k=1)
        # Absolute values
        abs_vals = np.abs(A[off_diag_indices])
        max_idx = np.argmax(abs_vals)
        p = off_diag_indices[0][max_idx]
        q = off_diag_indices[1][max_idx]
        a_pp = A[p, p]
        a_qq = A[q, q]
        a_pq = A[p, q]

        # Next compute the rotation angle
        if np.abs(a_pq) < 1e-14: # cCase where already negligible
            pass
        else:

            theta = 0.5 * np.arctan2(2*a_pq, (a_pp - a_qq)) # Formula(s:)
            c = np.cos(theta)
            s = np.sin(theta)
            # Apply the rotation & update diagonal elements
            A[p, p] = c*c*a_pp - 2*s*c*a_pq + s*s*a_qq
            A[q, q] = s*s*a_pp + 2*s*c*a_pq + c*c*a_qq
            A[p, q] = 0.0
            A[q, p] = 0.0

            # Update the rest
            for r in range(N):
                if r != p and r != q:
                    a_rp = A[r, p]
                    a_rq = A[r, q]
                    A[r, p] = c*a_rp - s*a_rq
                    A[p, r] = A[r, p]
                    A[r, q] = s*a_rp + c*a_rq
                    A[q, r] = A[r, q]

        # Finally heck convergence by off-diagonal norm
        off_norm = off_diagonal_norm(A)
        if off_norm < tol or iteration_count > max_iterations:
            break

    return A, iteration_count # Return

def tridiagonal_jacobi_method(A, tol=1e-12, max_iterations=10000): # limit iterations for time constraints
    #Tridiagonalized Jacobi by reducing A to tridiagonal form T and apply standard Jacobi on T
    T = householder_tridiagonal(A)  # from step 2
    return jacobi_method(T, tol=tol, max_iterations=max_iterations)

# Test and debug
'''B = generate_symmetric_matrix(5)
print(B)
H = householder_tridiagonal(B)
print(H)
J, c = jacobi_method(B)
print(J)
print(c)
T,cc = tridiagonal_jacobi_method(B)
print(T)
print(cc)'''

if __name__ == "__main__": # Run the code

    N_values = [4, 8, 16, 32, 64, 128] # By iterating over multiple values
    tol = 1e-12 # Tolerance set
    results = [] # Initialize an array to store the results

    for N in N_values: # Iterate

        A = generate_symmetric_matrix(N) # Generate a matrix for each N
        _, iter_std = jacobi_method(A, tol=tol) # Apply Standard Jacobi
        _, iter_tri = tridiagonal_jacobi_method(A, tol=tol) # Apply Tridiagonalized Jacobi
        results.append((N, iter_std, iter_tri)) # Store the results at the end of our results array

    # Print out results in terminal/Jupyter notebook
    print("N | Standard Jacobi Iters | Tridiagonal Jacobi Iters")
    print("--|-----------------------|--------------------------") # Format for clarity
    for (N, iter_std, iter_tri) in results:
        print(f"{N:2d} | {iter_std:21d} | {iter_tri:24d}")


N | Standard Jacobi Iters | Tridiagonal Jacobi Iters
--|-----------------------|--------------------------
 4 |                    20 |                       18
 8 |                   120 |                      105
16 |                   635 |                      500
32 |                  4646 |                     2154
64 |                 10001 |                     7115
128 |                 10001 |                    10001
